# New confirmed cases of COVID-19 and deaths each day

In [2]:
# TO DO: create setup.py and .py file with functions

In [3]:
import datetime
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets

In [12]:
nPerPopulation = 100000
fromTotalCases = 0
nDaysAverage = 7

base_url = "https://opendata.ecdc.europa.eu/covid19/casedistribution/csv"
df = pd.read_csv(base_url)

# sort from the oldest time data to the most recent
df['dateRep' ] = pd.to_datetime(df.dateRep, format='%d/%m/%Y')
df = df.sort_values(by=['countriesAndTerritories','dateRep'])

df_grouped = df.groupby('countriesAndTerritories')

In [13]:
def getNDayAverage(values, nDays):
    averaged = []
    for i, value in enumerate(values):
        dayIndex = 1
        toAverage = []
        while (i - dayIndex > 0) & (dayIndex <= nDays):
            toAverage.append(values[i - dayIndex])
            dayIndex += 1
        if len(toAverage) > 0:
            averaged.append(np.mean(toAverage))
    return np.array(averaged) 

def prepareData(df):
    # TO DO data as an dict???
    data = {}
    countryDict = {}

    for name, group in df:
        totalCases = 0
        cases = []
        deaths = []
        dates = []

        for i, row in group.iterrows():
            if totalCases <= fromTotalCases:
                totalCases += row['cases'] 
            if totalCases >= fromTotalCases:
                cases.append(row['cases'])
                deaths.append(row['deaths'])
                dates.append(row['dateRep'])

        data[name.replace("_", " ")] = { 
            "casesAveraged": getNDayAverage(cases, nDaysAverage),
            "deathsAveraged": getNDayAverage(deaths, nDaysAverage),
            "dates": dates, 
            "popData2018": group["popData2018"].iloc[0]
        }
        countryDict[name.replace("_", " ")] = name
    return data, countryDict

data, countryDict = prepareData(df_grouped)
selectedCountries = ["Czechia", "Germany", "Switzerland", "Austria","Italy", "Germany", "France", "Sweden","Belgium","United States of America","Spain"]
fig = go.Figure()

## Optional interactive checkbox for countries
When not run, default list of countries is used instead

In [14]:
# simple checkbox for countries
# TO DO: add select all, select none, automatically redraw graph
names = []
checkbox_objects = []
selectedDefaultCountries = selectedCountries

def select_data(**kwargs):
    selectedCountries.clear()

    for key in kwargs:
        if kwargs[key] is True:
            selectedCountries.append(key)
            
for country in countryDict:
    defaultValue = True if country in selectedDefaultCountries else False
    checkbox_objects.append(widgets.Checkbox(value=defaultValue, description=country))
    names.append(country)

arg_dict = {names[i]: checkbox for i, checkbox in enumerate(checkbox_objects)}
ui = widgets.VBox(children=checkbox_objects,layout=widgets.Layout(flex_flow='row wrap'))

selectedCountries = []

out = widgets.interactive_output(select_data, arg_dict)
display(ui, out)

Output()

## Show figure
You have to manually run the bellow cell after each change when interactive checkbox for countries is used

In [15]:
fig = go.Figure()
for countryName, countryData in data.items():
#     if countryData["popData2018"] > 1000000:
    if countryName in selectedCountries:
        fig.add_trace(go.Scatter(
            x=countryData["deathsAveraged"]*nPerPopulation/countryData["popData2018"], 
            y=countryData["casesAveraged"]*nPerPopulation/countryData["popData2018"],
            line_shape='spline',
            name=countryName,
            hovertemplate =
                '<b>{}</b>'.format(countryName)+
                '<br>Cases: %{y:.2f}'+
                '<br>Deaths: %{x:.2f}'+
                '<br>%{text}',
            text = [f"Date: {countryData['dates'][i].strftime('%b %d %Y')}<br> {i} days after first 10 confirmed cases" for i in range(countryData["casesAveraged"].shape[0])],
        ))
fig.update_layout(title=f"New confirmed cases and deaths each day ({nDaysAverage}-day-average), per {nPerPopulation} population",
                   xaxis_title='Deaths',
                   yaxis_title='Confirmed Cases')
fig.show()

In [8]:
# TO DO: Choose entire continet e.g. Africa

## Export chart to .html

In [11]:
fig.write_html("cases-deaths.html")